In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from itertools import combinations

import numpy as np
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr
from sklearn.metrics import f1_score
from statsmodels.stats import inter_rater

import src
import src.bert.dataset.strategies as strategy
import src.db.connect
from src.bert.dataset import PBertDataset

In [ ]:
dataset = PBertDataset.from_disk(
    path=src.PATH / "data/bert/dataset.csv.zip",
    exclude_coders=[],
    label_strategy=strategy.MLMin1PopIdeol(output_fmt="single_task"),
)

In [ ]:
def pairwise_f1_agreement(df):
    scores = []
    for i, j in combinations(df.columns, r=2):
        coder1 = df[i].apply(pd.Series).to_numpy()
        coder2 = df[j].apply(pd.Series).to_numpy()
        score = f1_score(coder1, coder2, average="binary")
        scores.append(score)

    return np.mean(scores)

In [ ]:
base = importr("base")
irr = importr("irr")

In [ ]:
def pairwise_f1_agreement(df):
    scores = []
    for i, j in combinations(df.columns, r=2):
        coder1 = df[i]
        coder2 = df[j]
        score = f1_score(coder1, coder2, average="binary")
        scores.append(score)

    return np.mean(scores)

In [ ]:
def calculate_kappa(dataset, strategy, coders):
    dataset.valid_coders = coders
    dataset.strategy = strategy
    dataset.apply_label_strategy()

    df = dataset.df_labels.copy()
    raw_data = pd.DataFrame(df.labels.tolist(), columns=dataset.coders, index=df.id)
    f1_agreement = pairwise_f1_agreement(raw_data)
    raw_data = raw_data.to_numpy()

    with (ro.default_converter + numpy2ri.converter).context():
        r_mat = ro.conversion.get_conversion().py2rpy(raw_data)

    out = irr.kappam_fleiss(r_mat, exact=False)
    kappa = np.array(out.rx("value"))[0][0]

    out = irr.agree(r_mat, tolerance=0)
    agree = np.array(out.rx("value"))[0][0]

    out = irr.kripp_alpha(base.t(r_mat), method="nominal")
    alpha = np.array(out.rx("value"))[0][0]

    return round(kappa, 3), round(alpha, 3), round(agree, 1), round(f1_agreement, 3)

In [ ]:
coder_sets = {"all_coders": dataset.all_coders}

strategies = [
    # strategy.MCPopBin(),
    strategy.PopEliteBin(),
    strategy.PopCentrBin(),
    strategy.IdeolLeftBin(),
    strategy.IdeolRightBin(),
    # strategy.MCPopBinIdeol(),
    # strategy.MCPopIdeol(),
]

# Kappa Values


In [ ]:
rows = []
for strat in strategies:
    for set_name, coders in coder_sets.items():
        kappa, alpha, agree, f1 = calculate_kappa(dataset, strat, coders)
        rows.append((type(strat).__name__, set_name, kappa, f1, alpha, agree))

pd.DataFrame(
    rows,
    columns=[
        "strategy",
        "coder_set",
        "fleiss_kappa",
        "f1",
        "krippendorff_alpha",
        "agree",
    ],
).set_index(["strategy", "coder_set"])

,,fleiss_kappa,f1,krippendorff_alpha,agree
strategy,coder_set,,,,
PopEliteBin,all_coders,0.410,0.496,0.410,65.8
PopCentrBin,all_coders,0.244,0.262,0.244,81.8
IdeolLeftBin,all_coders,0.355,0.378,0.355,84.5
IdeolRightBin,all_coders,0.364,0.383,0.364,91.6


# Conflicting Left / Right


In [ ]:
def conflicting_samples_count(dataset, coders):
    # all coders
    dataset.valid_coders = coders
    dataset.strategy = strategy.MLIdeol()
    dataset.apply_label_strategy()

    subset = len(dataset.df_labels[dataset.df_labels.vote == tuple([1, 1])])
    total = len(dataset.df_labels)

    return subset, total


subset, total = conflicting_samples_count(dataset, coder_sets["all_coders"])
print(
    f"{subset} out of {total} samples {subset / total:.4%} have a positive majority vote on "
    "both dimensions. (all coders)"
)

subset, total = conflicting_samples_count(dataset, coder_sets["reduced_coders"])
print(
    f"{subset} out of {total} samples {subset / total:.4%} have a positive majority vote on "
    "both dimensions. (reduced coders)"
)

In [ ]:
def conflicting_codings_count(dataset, coders):
    username = dataset.df.username.isin(coders)
    subset = len(dataset.df[username & dataset.df.left & dataset.df.right])
    total = len(dataset.df[username])

    return subset, total


subset, total = conflicting_codings_count(dataset, coder_sets["all_coders"])
print(
    f"{subset} out of {total} codings {subset / total:.4%} have are positive on both dimensions. "
    "(all coders)"
)

subset, total = conflicting_codings_count(dataset, coder_sets["reduced_coders"])
print(
    f"{subset} out of {total} codings {subset / total:.4%} have are positive on both dimensions. "
    "(reduced coders)"
)

In [ ]:
# Number of codings with left OR right but no populism


def left_right_no_pop_codings(dataset, coders):
    username = dataset.df.username.isin(coders)
    subset = len(
        dataset.df[
            username & ~(dataset.df.elite | dataset.df.centr) & (dataset.df.left | dataset.df.right)
        ]
    )
    total = len(dataset.df[username])
    return subset, total


subset, total = left_right_no_pop_codings(dataset, coder_sets["all_coders"])
print(
    f"{subset} out of {total} codings {subset / total:.4%} have left OR right but not populism. "
    "(all coders)"
)
subset, total = left_right_no_pop_codings(dataset, coder_sets["reduced_coders"])
print(
    f"{subset} out of {total} codings {subset / total:.4%} have left OR right but not populism. "
    "(reduced coders)"
)

In [ ]:
def left_right_no_pop_samples(dataset, coders):
    dataset.valid_coders = coders
    dataset.strategy = strategy.MLPopBinIdeol()
    dataset.apply_label_strategy()

    subset = len(
        dataset.df_labels[
            dataset.df_labels.vote.apply(lambda x: (x[0] == 0) and (x[1] == 1 or x[2] == 1))
        ]
    )
    total = len(dataset.df_labels)
    return subset, total


subset, total = left_right_no_pop_samples(dataset, coder_sets["all_coders"])
print(
    f"{subset} out of {total} samples {subset / total:.4%} have left OR right but not populism. "
    "(all coders)"
)

subset, total = left_right_no_pop_samples(dataset, coder_sets["reduced_coders"])
print(
    f"{subset} out of {total} samples {subset / total:.4%} have left OR right but not populism. "
    "(reduced coders)"
)